# Assignment Module 2: Product Classification

The goal of this assignment is to implement a neural network that classifies smartphone pictures of products found in grocery stores. The assignment will be divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.


## Preliminaries: the dataset

The dataset you will be using contains natural images of products taken with a smartphone camera in different grocery stores:

<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Granny-Smith.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Pink-Lady.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Lemon.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Banana.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Vine-Tomato.jpg" width="150">
</p>
<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Yellow-Onion.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Green-Bell-Pepper.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Arla-Standard-Milk.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Oatly-Natural-Oatghurt.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Alpro-Fresh-Soy-Milk.jpg" width="150">
</p>

The products belong to the following 43 classes:
```
0.  Apple
1.  Avocado
2.  Banana
3.  Kiwi
4.  Lemon
5.  Lime
6.  Mango
7.  Melon
8.  Nectarine
9.  Orange
10. Papaya
11. Passion-Fruit
12. Peach
13. Pear
14. Pineapple
15. Plum
16. Pomegranate
17. Red-Grapefruit
18. Satsumas
19. Juice
20. Milk
21. Oatghurt
22. Oat-Milk
23. Sour-Cream
24. Sour-Milk
25. Soyghurt
26. Soy-Milk
27. Yoghurt
28. Asparagus
29. Aubergine
30. Cabbage
31. Carrots
32. Cucumber
33. Garlic
34. Ginger
35. Leek
36. Mushroom
37. Onion
38. Pepper
39. Potato
40. Red-Beet
41. Tomato
42. Zucchini
```

The dataset is split into training (`train`), validation (`val`), and test (`test`) set.

The following code cells download the dataset and define a `torch.utils.data.Dataset` class to access it. This `Dataset` class will be the starting point of your assignment: use it in your own code and build everything else around it.

In [ ]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

In [4]:
from pathlib import Path
from PIL import Image
from torch import Tensor
from torch.utils.data import Dataset
from typing import List, Tuple

In [5]:
class GroceryStoreDataset(Dataset):

    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split
        self.paths, self.labels = self.read_file()

        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths = []
        labels = []

        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                # path, fine-grained class, coarse-grained class
                path, _, label = line.replace("\n", "").split(", ")
                paths.append(path), labels.append(int(label))

        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it on `GroceryStoreDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split of **around 60%**. You are free to achieve that however you want, except for a few rules you must follow:

- You **cannot** simply instantiate an off-the-self PyTorch network. Instead, you must construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you **cannot** use e.g. `torchvision.models.alexnet`.

- Justify every *design choice* you make. Design choices include network architecture, training hyperparameters, and, possibly, dataset preprocessing steps. You can either (i) start from the simplest convolutional network you can think of and add complexity one step at a time, while showing how each step gets you closer to the target ~60%, or (ii) start from a model that is already able to achieve the desired accuracy and show how, by removing some of its components, its performance drops (i.e. an *ablation study*). You can *show* your results/improvements however you want: training plots, console-printed values or tables, or whatever else your heart desires: the clearer, the better.

Don't be too concerned with your network performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded **more** points than a poorly experimentally validated model with higher accuracy.

## Part 2: fine-tune an existing network

Your goal is to fine-tune a pretrained **ResNet-18** model on `GroceryStoreDataset`. Use the implementation provided by PyTorch, do not implement it yourselves! (i.e. exactly what you **could not** do in the first part of the assignment). Specifically, you must use the PyTorch ResNet-18 model pretrained on ImageNet-1K (V1). Divide your fine-tuning into two parts:

1. First, fine-tune the Resnet-18 with the same training hyperparameters you used for your best model in the first part of the assignment.
1. Then, tweak the training hyperparameters in order to increase the accuracy on the validation split of `GroceryStoreDataset`. Justify your choices by analyzing the training plots and/or citing sources that guided you in your decisions (papers, blog posts, YouTube videos, or whatever else you find enlightening). You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split **between 80 and 90%**.

---
---

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import numpy as np
import random
from typing import List, Tuple
from torch import Tensor
from pathlib import Path
from torchvision import transforms as T, datasets
from PIL import Image
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

def fix_random(seed: int) -> None:
    """
        Fix all the possible sources of randomness.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Device: cpu


## Data preprocessing

In [14]:
NUM_CLASSES = 43
# input_width = min([item[0][0].shape[1] for item in train_dset])
# input_height = min([item[0][0].shape[0] for item in train_dset])
INPUT_WIDTH = 348
INPUT_HEIGHT = 348
VALIDATION_SPLIT_SIZE = 0.2
BATCH_SIZE = 64

tsfms = T.Compose([
    T.ToTensor(),
    T.CenterCrop((INPUT_HEIGHT, INPUT_WIDTH)) # Some images are not the same size
])
train_dset = GroceryStoreDataset(split="train", transform=tsfms)
test_dset = GroceryStoreDataset(split="test", transform=tsfms)
val_dset = GroceryStoreDataset(split="val", transform=tsfms)

#split_dsets = random_split(train_dset, [1 - VALIDATION_SPLIT_SIZE, VALIDATION_SPLIT_SIZE])
#train_subdset = split_dsets[0]
#val_subdset = split_dsets[1]

train_dl = DataLoader(train_dset, BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_dset, BATCH_SIZE)
test_dl = DataLoader(test_dset, BATCH_SIZE)

## Part 1

### Basic Model

Each Convolutional block in the network has the following structure:
1. 2D Convolution
2. SiLU Activation Function
3. Max Pooling
4. Batch Normalization
5. Dropout

In [15]:
def conv_out_shape(h_w, padding, kernel_size, stride):
    from math import floor
    h, w = h_w
    h = floor((h - kernel_size + (2 * padding)) / stride) + 1
    w = floor((w - kernel_size + (2 * padding)) / stride) + 1
    return h, w


def convBlock(
    input_shape, out_channels, 
    conv_kernel_size, conv_padding, conv_stride, 
    pool_size, pool_stride,
    dropout_prob,
    activation: nn.Module = nn.Identity()
):
    block = nn.Sequential(
        nn.Conv2d(
            in_channels = input_shape[0], 
            out_channels = out_channels, 
            kernel_size = conv_kernel_size, 
            stride = conv_stride, 
            padding = conv_padding
        ),
        activation,
        nn.MaxPool2d(kernel_size=pool_size, stride=pool_stride),
        nn.BatchNorm2d(out_channels),
        nn.Dropout(p=dropout_prob),
    )

    if conv_padding == "same": actual_conv_padding = conv_kernel_size // 2
    elif conv_padding == "valid": actual_conv_padding = 0
    else: actual_conv_padding = conv_padding

    shape = conv_out_shape((input_shape[2], input_shape[1]), actual_conv_padding, conv_kernel_size, conv_stride)
    shape = conv_out_shape(shape, 0, pool_size, pool_stride)

    return block, (out_channels, *shape)


### First Model

In [18]:
class FirstModel(nn.Module):
    def __init__(self, input_dim, n_classes):
        super().__init__()
        
        self.stem_block = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=3, padding="valid"),
            nn.SiLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.05),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=2, padding="valid"),
            nn.SiLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(p=0.05)
        )
        
        output_shape = conv_out_shape((INPUT_HEIGHT, INPUT_WIDTH), padding=0, kernel_size=7, stride=3)
        output_shape = conv_out_shape(output_shape, padding=0, kernel_size=5, stride=2)
        
        self.conv_block_1, output_shape = convBlock(
            input_shape=(128, output_shape[1], output_shape[0]), 
            out_channels=256, 
            conv_kernel_size=3, conv_padding="same", conv_stride=1,
            pool_size=2, pool_stride=2,
            dropout_prob=0.05,
            activation=nn.SiLU()
        )
        
        self.conv_block_2, output_shape = convBlock(
            input_shape=output_shape, 
            out_channels=256, 
            conv_kernel_size=3, conv_padding="same", conv_stride=1,
            pool_size=2, pool_stride=2,
            dropout_prob=0.05,
            # activation=nn.SiLU()
        )

        self.conv_block_3, output_shape = convBlock(
            input_shape=output_shape, 
            out_channels=512, 
            conv_kernel_size=3, conv_padding="same", conv_stride=1,
            pool_size=2, pool_stride=2,
            dropout_prob=0.05,
            activation=nn.SiLU()
        )
        
        self.linear_block = nn.Sequential(
            nn.Linear(512, 512),
            nn.SiLU(),
            nn.Dropout(p=0.05),
            nn.Linear(512, 1024),
            nn.SiLU(),
            nn.Dropout(p=0.05),
            nn.Linear(1024, 1024),
            nn.SiLU(),
            nn.Dropout(p=0.05),
            nn.Linear(1024, n_classes)
        )        
        
    def forward(self, x):
        x = self.stem_block(x)
        
        x1 = self.conv_block_1(x)
        x1_residual = x1
        x1_residual = F.max_pool2d(x1_residual, kernel_size=2, stride=2)
        
        x2 = self.conv_block_2(x1)
        x2 = x2 + x1_residual
        
        x2 = F.silu(x2) # Apply the SiLU activation function only after the residual connection
        
        x3 = self.conv_block_3(x2)
        
        # Before flattening the tensor, to further reduce the parameters
        # we use adaptive average pooling
        x = F.adaptive_avg_pool2d(x3, (1, 1))
        x = torch.flatten(x, 1)
        
        x = self.linear_block(x)
        return x


In [11]:

import os
import json


def train(
    model, 
    loss_fn, 
    optimizer, 
    train_dl, 
    val_dl, 
    device, 
    n_epochs, 
    checkpoint_path = None,
    logging_path = None,
    random_seed = 42
):
    fix_random(random_seed)
    if checkpoint_path is not None: os.makedirs(checkpoint_path, exist_ok=True)
    if logging_path is not None: os.makedirs(logging_path, exist_ok=True)

    history = []
    model = model.to(device)
    loss_fn = loss_fn.to(device)


    for epoch in tqdm(range(n_epochs)):
        # Train
        model.train()
        batch_train_accuracies = []
        batch_train_losses = []
        batch_train_times = []
        for i, (images, labels) in enumerate(train_dl):
            start_time = torch.cuda.Event(enable_timing=True)
            end_time = torch.cuda.Event(enable_timing=True)
            images, labels = images.to(device), labels.to(device)

            start_time.record()
            optimizer.zero_grad()
            pred_logits = model(images)
            loss = loss_fn(pred_logits, labels)
            loss.backward()
            optimizer.step()
            end_time.record()
            
            _, preds = torch.max(pred_logits, 1)
            batch_train_accuracies.append((preds == labels).sum().item() / labels.size(0))
            batch_train_losses.append(loss.item())
            torch.cuda.synchronize()
            batch_train_times.append(start_time.elapsed_time(end_time))


        # Validation
        batch_val_accuracies = []
        batch_val_losses = []
        
        model.eval()
        with torch.no_grad():
            for images, labels in val_dl:
                images, labels = images.to(device), labels.to(device)
                preds_logits = model(images)
                _, pred = torch.max(preds_logits, 1)
                batch_val_accuracies.append((pred == labels).sum().item() / labels.size(0))
                batch_val_losses.append(loss_fn(preds_logits, labels).item())

        print(f"Epoch {epoch}, "
                f"train_acc: {np.mean(batch_train_accuracies):.4f}, "
                f"val_acc: {np.mean(batch_val_accuracies):.4f}, "
                f"train_loss: {np.mean(batch_val_losses):.4f}, "
                f"val_loss: {np.mean(batch_train_losses):.4f}")
        
        # Checkpoint
        if checkpoint_path is not None:
            torch.save({
                "state_dict": model.state_dict(),
                "architecture": str(model)
            }, os.path.join(checkpoint_path, f"epoch{epoch+1}.ckp.pt"))

        # Logging
        epoch_logs = {
            "epoch": epoch + 1,
            "batch_train_accuracies": batch_train_accuracies,
            "batch_train_losses": batch_train_losses,
            "batch_train_times": batch_train_times,
            "batch_val_accuracies": batch_val_accuracies,
            "batch_val_losses": batch_val_losses
        }
        if logging_path is not None:
            with open(os.path.join(logging_path, f"epoch{epoch+1}.log.json"), "w") as f:
                json.dump(epoch_logs, f, indent=3)
        history.append(epoch_logs)
            
    return history

model = FirstModel((INPUT_HEIGHT, INPUT_WIDTH), NUM_CLASSES).to(device)
print(model(train_dset[0][0].unsqueeze(0).to(device)))

logs = train(
    model = model,
    loss_fn = nn.CrossEntropyLoss(),
    optimizer = optim.Adam(model.parameters(), lr=2e-4),
    train_dl = train_dl, 
    val_dl = val_dl,
    device = device,
    n_epochs = 2,
    random_seed = 42,
    checkpoint_path = "./checkpoints",
    logging_path = "./logs"
)


tensor([[-0.0260,  0.0140, -0.0099, -0.0167,  0.0186,  0.0076,  0.0155,  0.0179,
         -0.0200, -0.0182, -0.0184,  0.0203, -0.0137, -0.0005,  0.0190, -0.0046,
          0.0058, -0.0019,  0.0084,  0.0062, -0.0180, -0.0326, -0.0286,  0.0110,
         -0.0247,  0.0024, -0.0226,  0.0223, -0.0044,  0.0349,  0.0149, -0.0302,
          0.0167, -0.0086,  0.0252, -0.0065,  0.0190, -0.0114, -0.0264,  0.0100,
         -0.0338, -0.0142, -0.0167]], grad_fn=<AddmmBackward0>)


  0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: Tried to instantiate dummy base class Event